In [1]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets, evaluation
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import torch
import random
import torch

from google.colab import drive
drive.mount("/content/gdrive")

import h5py
import os
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=f7c4f0b58b23363ac77192ad5bc187cfd3861ddaef61396d234dc0092d1da2d8
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Mounted at /content/gdrive


In [2]:
# train_df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

# Load the test dataset into a pandas dataframe.
"""
eval_df = pd.read_csv('https://raw.githubusercontent.com/sanromra/imdb_data/main/test_IMDB.tsv', delimiter='\t', header=None)

text_col=eval_df.columns.values[0] 
category_col=eval_df.columns.values[1]

x_eval = eval_df[text_col].values.tolist()[:10]
y_eval = eval_df[category_col].values.tolist()[:10]
"""
dataset = "dbpedia"

if dataset == "yahoo":
  test_data =  h5py.File(os.path.join("gdrive/MyDrive/DL_Experiments/TextExperiments/Federated_Learning/result/yahoo_silos", 'processed_test.h5'), 'r')
else:
  print("Dbpedia loaded!")
  test_data =  h5py.File(os.path.join("gdrive/MyDrive/DL_Experiments/TextExperiments/data/new_data/dbpedia", 'processed_test.h5'), 'r')

x_eval = torch.tensor(np.array(test_data['text_embeddings']))
y_eval = torch.tensor(np.array(test_data['labels']))

Dbpedia loaded!


In [ ]:
#@title SetFit
st_model = 'paraphrase-mpnet-base-v2' #@param ['paraphrase-mpnet-base-v2', 'all-mpnet-base-v1', 'all-mpnet-base-v2', 'stsb-mpnet-base-v2', 'all-MiniLM-L12-v2', 'paraphrase-albert-small-v2', 'all-roberta-large-v1']
num_training = 32 #@param ["8", "16", "32", "54", "128", "256", "512"] {type:"raw"}

set_seed(0)

orig_model = SentenceTransformer(st_model)

X_eval_noFT = orig_model.encode(x_eval, show_progress_bar = True, device = 'mps')

KeyboardInterrupt: ignored

In [3]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 256
train_loaders = []
num_silos = 10

if dataset == "yahoo":
  train_data = h5py.File(os.path.join("gdrive/MyDrive/DL_Experiments/TextExperiments/data/new_data/yahoo", 'processed_train.h5'), 'r')
  exp_results = torch.load('gdrive/MyDrive/DL_Experiments/TextExperiments/Federated_Learning/result/yahoo/whole/res_DM_yahoo_MLP_10ipc.pt', map_location = 'cpu')
else:
  print("Loaded dbpedia!")
  train_data = h5py.File(os.path.join("gdrive/MyDrive/DL_Experiments/TextExperiments/data/new_data/dbpedia", 'processed_train.h5'), 'r')
  exp_results = torch.load('gdrive/MyDrive/DL_Experiments/TextExperiments/Federated_Learning/result/dbpedia/whole/res_DM_dbpedia_MLP_10ipc.pt', map_location = 'cpu')

x_train = torch.tensor(np.array(train_data['text_embeddings']))
y_train = torch.tensor(np.array(train_data['labels']))

trainloader_onesilo = None
length = y_train.size(dim=0)
classes = 10 if dataset == "yahoo" else 14
print("CLASSES: {}".format(classes))
total = 0
"""
for i in range(num_silos):
#     exp_results = torch.load('/Users/gauravnuti/Downloads/10ipc/10ipc_silo'+str(i)+ '.pt', map_location = 'mps')
    
    

    
    new_x_train = []
    new_y_train = []

    for j in range(classes):
      idx = torch.where(y_train == j, 1, 0)
      idx = torch.nonzero(idx)
      samples_per_silo = idx.size(dim=0) // 10

      start = samples_per_silo*i
      end = samples_per_silo*(i+1)
      idx_class = idx[start:end]
      new_x_train.append(x_train[idx_class])
      new_y_train.append(y_train[idx_class])

    print(torch.cat(new_x_train, dim=0).shape)
    
    new_x_train = np.array(torch.cat(new_x_train, dim=0)).squeeze()
    total += new_x_train.shape[0]
    new_y_train = np.array(torch.cat(new_y_train, dim=0)).squeeze()
    #x_train = torch.tensor(new_x_train)
    #y_train = torch.tensor(new_y_train)

    trainset = TensorDataset(torch.Tensor(new_x_train),torch.Tensor(new_y_train)) # create your datset
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
    if i == 0:
      trainloader_onesilo = trainloader
    train_loaders.append(trainloader)
    

x_train_split = [[] for k in range(classes)]
y_train_split = [[] for k in range(classes)]
for i in range(classes):
  x_train_class = x_train[y_train == i]
  y_train_class = y_train[y_train == i]
  centers = exp_results['data'][0][0][exp_results['data'][0][1] == i]
  distances = torch.cdist(x_train_class, centers)
  closest_centers = torch.argmin(distances, dim=1)
  for j in range(classes):
    x_train_split[j].append(x_train_class[closest_centers == j])
    y_train_split[j].append(y_train_class[closest_centers == j])
  
  #print('silo s = %d: %d real data'%(i, y_train_class[closest_centers == j].shape[0]))

#new_x_train = np.array(torch.cat(torch.Tensor(new_x_train), dim=0)).squeeze()
#new_y_train = np.array(torch.cat(torch.Tensor(new_y_train), dim=0)).squeeze()


#torch.cat(tuple(x_train_federated), dim = 0)
total = 0
for c in range(num_silos):
  new_x_train = x_train_split[c]
  new_y_train = y_train_split[c]
  new_x_train = np.array(torch.cat(new_x_train, dim=0)).squeeze()
  new_y_train = np.array(torch.cat(new_y_train, dim=0)).squeeze()
  print(new_x_train.shape)
  total += new_x_train.shape[0]
  xt = torch.tensor(new_x_train)
  yt = torch.tensor(new_y_train)
  
  trainset = TensorDataset(xt,yt) # create your datset
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
  if i == 0:
    trainloader_onesilo = trainloader
  train_loaders.append(trainloader)
"""
num_classes = classes
num_silos = num_classes ## Equal to num silos
x_train_split = [[] for i in range(num_silos)]
y_train_split = [[] for i in range(num_silos)]
shards_per_class = 2
shards_per_silo = (shards_per_class * num_classes)//num_silos
order = [i//shards_per_silo for i in range(num_silos * shards_per_silo)]
order = random.sample(order, k = num_silos * shards_per_silo)

count = 0
for i in range(num_classes):
  x_train_class = x_train[y_train == i]
  y_train_class = y_train[y_train == i]
  x_train_class_split = torch.chunk(x_train_class, shards_per_class, dim=0)
  y_train_class_split = torch.chunk(y_train_class, shards_per_class, dim=0)
  for i1 in range(len(x_train_class_split)):
    x_train_split[order[count]].append(x_train_class_split[i1])
    y_train_split[order[count]].append(y_train_class_split[i1])
    count += 1

for c in range(num_silos):
  new_x_train = x_train_split[c]
  new_y_train = y_train_split[c]
  new_x_train = np.array(torch.cat(new_x_train, dim=0)).squeeze()
  new_y_train = np.array(torch.cat(new_y_train, dim=0)).squeeze()
  print(new_x_train.shape)
  total += new_x_train.shape[0]
  xt = torch.tensor(new_x_train)
  yt = torch.tensor(new_y_train)
  
  trainset = TensorDataset(xt,yt) # create your datset
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
  if i == 0:
    trainloader_onesilo = trainloader
  train_loaders.append(trainloader)
print("Real total: {}\t Obtained total: {}".format(x_train.shape[0], total))

Loaded dbpedia!
CLASSES: 14
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
(40000, 768)
Real total: 560000	 Obtained total: 560000


In [4]:
valset = TensorDataset(torch.Tensor(x_eval),torch.Tensor(y_eval)) # create your datset
valloader = torch.utils.data.DataLoader(valset, batch_size=16, num_workers=0)

In [5]:
device = "cuda"
def epoch(mode, dataloader, net, optimizer, criterion):
    loss_avg, acc_avg, num_exp = 0, 0, 0
    #print("Loss, acc, num")
    net = net.to(device)
    #print("net to device")
    criterion = criterion.to(device)
    #print("criterion to device")

    if mode == 'train':
        net.train()
        #print("net train")
    else:
        net.eval()
        #print("net eval")

    for i_batch, datum in enumerate(dataloader):
        img = datum[0].float().to(device)
        #print("sample to device")
        #print(img)
        lab = datum[1].long().to(device)
        #print("label to device")
        #print(lab)
        n_b = lab.shape[0]
        #print("n_b")

        output = net(img)
        #print("get output")
        #print("Output:\n{}".format(output))
        #print("Label:\n{}".format(lab.cpu().data.numpy()))
        loss = criterion(output, lab)
        #print("get loss")
        #print("Output:\n{}".format(output))
        #print(lab.cpu().data.numpy())
        acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
        #print("get accuracy")

        loss_avg += loss.item()*n_b
        #print("get avg loss")
        acc_avg += acc
        #print("get avg acc")
        num_exp += n_b
        #rint("num_exp")

        if mode == 'train':
            optimizer.zero_grad()
            #print("zerograd")
            loss.backward()
            #print("backprop")
            optimizer.step()
            #print("grad update")

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

In [6]:
import torch.nn as nn
embed_dim = 768
hidden_dim = 512
num_classes = 14

In [7]:
device = 'cuda'
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
models = []

In [ ]:
# import copy
# net1 = copy.deepcopy(net)

In [ ]:
# net2 = nn.Sequential(nn.Linear(embed_dim, hidden_dim),
#             nn.LeakyReLU(0.2),
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.LeakyReLU(0.2),
#             nn.Linear(hidden_dim, num_classes))

In [ ]:
# with torch.no_grad():
#     net2[0].weight.data = net1[0].weight.data.clone() * 2
#     net2[0].bias.data = net1[0].bias.data.clone() * 2

In [ ]:
# type(net2[0]) == torch.nn.modules.linear.Linear

In [8]:
# net2[0].weight.data.shape
print("Num classes = {}".format(num_classes))

Num classes = 14


In [10]:
import copy

net = nn.Sequential(nn.Linear(embed_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            #nn.Linear(hidden_dim, hidden_dim),
            #nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, num_classes))
net.to("cuda")

ep_per_round = 10
num_communication_rounds = 5
Epoch = num_communication_rounds * ep_per_round
lr_schedule = [Epoch//2+1]
#lr_schedule = []
lr = 0.01
curr_epoch = 0

for comm_round in range(num_communication_rounds):
    models = [copy.deepcopy(net) for _ in range(num_silos)]
    if comm_round == 0:
      print(len(models))
    optimizers = [torch.optim.SGD(models[model_num].parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)\
     for model_num in range(num_silos)]#, weight_decay=0.0005)\
            
    for ep in range(curr_epoch, curr_epoch + ep_per_round):
        if ep in lr_schedule:
            lr *= 0.1
            optimizers = [torch.optim.SGD(models[model_num].parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)\
             for model_num in range(num_silos)]#, weight_decay=0.0005)\
        for model_num in range(num_silos):
            loss_train, acc_train = epoch('train', train_loaders[model_num], models[model_num], optimizers[model_num], criterion)
            loss, acc = epoch('test', valloader, models[model_num], optimizers[model_num], criterion)
            print(acc, ep, model_num)
    
    for layer in range(len(net)):
        with torch.no_grad():
            if(type(net[layer]) == torch.nn.modules.linear.Linear):
                net[layer].weight.data = torch.zeros(net[layer].weight.data.shape).to(device)
                net[layer].bias.data = torch.zeros(net[layer].bias.data.shape).to(device)
        for silo in range(num_silos):
            with torch.no_grad():
                if(type(net[layer]) == torch.nn.modules.linear.Linear):
                    net[layer].weight.data += models[silo][layer].weight.data.clone() * (1/num_silos)
                    net[layer].bias.data += models[silo][layer].bias.data.clone() * (1/num_silos)
    loss, acc = epoch('test', valloader, net, None, criterion)
    print(acc, comm_round)

14
0.1425 0 0
0.07142857142857142 0 1
0.1424 0 2
0.14277142857142858 0 3
0.14275714285714286 0 4
0.1410142857142857 0 5
0.14181428571428573 0 6
0.1399 0 7
0.14282857142857142 0 8
0.14277142857142858 0 9
0.14172857142857143 0 10
0.14281428571428573 0 11
0.14284285714285713 0 12
0.14282857142857142 0 13
0.14264285714285715 1 0
0.07142857142857142 1 1
0.1425 1 2
0.1427857142857143 1 3
0.14277142857142858 1 4
0.1414 1 5
0.14224285714285714 1 6
0.1405142857142857 1 7
0.14282857142857142 1 8
0.14277142857142858 1 9
0.14191428571428572 1 10
0.14281428571428573 1 11
0.14285714285714285 1 12
0.14284285714285713 1 13
0.14272857142857143 2 0
0.07142857142857142 2 1
0.1425857142857143 2 2
0.1427857142857143 2 3
0.14277142857142858 2 4
0.14145714285714286 2 5
0.14247142857142858 2 6
0.14074285714285714 2 7
0.14282857142857142 2 8
0.14277142857142858 2 9
0.142 2 10
0.14281428571428573 2 11
0.14285714285714285 2 12
0.14284285714285713 2 13
0.14275714285714286 3 0
0.07142857142857142 3 1
0.14258571428

In [ ]:
epoch('test', valloader, net, None, criterion)

(0.1252387542728867, 0.9706285714285714)

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)
print(trainloader_onesilo)
for ep in range(Epoch+1):
        loss_train, acc_train = epoch('train', trainloader_onesilo, net, optimizer, criterion)
        if ep in lr_schedule:
            lr *= 0.1
            optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)
        loss, acc = epoch('test', valloader, net, optimizer, criterion)
        print(acc)

0.2718
0.5191
0.6115
0.6361285714285714
0.647
0.6522714285714286
0.6580428571428572
0.6658428571428572
0.6736428571428571
0.6795714285714286
0.6833428571428571
0.6855142857142857
0.6869285714285714
0.6879571428571428
0.6886428571428571
0.6896857142857142
0.6904142857142858
0.6910714285714286
0.6916428571428571
0.6921428571428572
0.6924142857142858
0.6927285714285715
0.6931285714285714
0.6935428571428571
0.6938857142857143
0.6941714285714286
0.6944428571428571
0.6944571428571429
0.6944571428571429
0.6944571428571429
0.6944857142857143
0.6944857142857143
0.6944857142857143
0.6945142857142858
0.6945285714285714
0.6945285714285714
0.6945714285714286
0.6945857142857143
0.6946142857142857
0.6946714285714286
0.6947
0.6947285714285715
0.6947428571428571
0.6947571428571429
0.6948142857142857
0.6948285714285715
0.6948714285714286
0.6948857142857143
0.6949142857142857


KeyboardInterrupt: ignored